# Simple CES function using pybaum and dags

In [1]:
import dags

In [2]:
start_params = {
    "preferences": {
        "leisure_weight": 0.9,
        "ces": 0.5
    },
    "work": {
        "hourly_wage": 25,
        "hours": 2_000
    },
    "time_budget": 24 * 7 * 365,
    "consumption_floor": 3_000,
}

In [3]:
def utility(consumption, leisure, params):
    ɑ = params["preferences"]["leisure_weight"]
    ɣ = params["preferences"]["ces"]
    c = (1 - ɑ) ** (1 / ɣ) * consumption ** ((ɣ - 1) / ɣ)
    l = ɑ ** (1 / ɣ) * leisure ** ((ɣ - 1) / ɣ)
    return (c + l) ** (ɣ / (ɣ - 1))

def leisure(params):
    return params["time_budget"] - params["work"]["hours"]


def income(params):
    return params["work"]["hourly_wage"] * params["work"]["hours"]

def consumption(income, params):
    c_min = params["consumption_floor"]
    return income if income > c_min else c_min

def unrelated(working_hours):
    raise NotImplementedError()


In [4]:
model = dags.concatenate_functions(
    functions=[utility, unrelated, leisure, consumption, income],
    targets=["utility", "consumption"],
    return_type="dict"
)

model(params=start_params)

{'utility': 72177.39187992174, 'consumption': 50000}

# Optimize working hours using estimagic

In [5]:
from estimagic import maximize
from copy import deepcopy

### Version of *model* which only returns utility 

In [6]:
u = dags.concatenate_functions(
    functions=[utility, leisure, income, consumption],
    targets="utility"
)

### Set up problem

- Use above parameters as starting values
- Fix all parameters except for working hours
- Set bounds on working hours

In [7]:
def return_all_but_working_hours(params):
    out = deepcopy(params)
    del out["work"]["hours"]
    return out

In [8]:
result = maximize(
    criterion=u,
    params=start_params,
    constraints=[
        {
            "selector": return_all_but_working_hours, 
            "type": "fixed"
        },
    ],
    lower_bounds={"work": {"hours": 0}},
    upper_bounds={"work": {"hours": start_params["time_budget"]}},
    algorithm="scipy_lbfgsb",
    logging="log.db"
)

In [9]:
result

Maximize with 1 free parameters terminated successfully after 8 criterion evaluations, 8 derivative evaluations and 5 iterations.

The value of criterion improved from 72177.39187992174 to 72448.01512285274.

The scipy_lbfgsb algorithm reported: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL

Independent of the convergence criteria used by scipy_lbfgsb, the strength of convergence can be assessed by the following criteria:

                             one_step    five_steps 
relative_criterion_change   5.44e-09**   0.003729   
relative_params_change     0.0004982       0.4998   
absolute_criterion_change  0.0003941        270.2   
absolute_params_change        0.6642        666.3   

(***: change <= 1e-10, **: change <= 1e-8, *: change <= 1e-5. Change refers to a change between accepted steps. The first column only considers the last step. The second column considers the last five steps.)

### Look at results, income/consumption

In [10]:
result.params

{'preferences': {'leisure_weight': 0.9, 'ces': 0.5},
 'work': {'hourly_wage': 25.0, 'hours': 1333.048247238629},
 'time_budget': 61320.0,
 'consumption_floor': 3000.0}

In [11]:
model(params=result.params)

{'utility': 72448.01512285274, 'consumption': 33326.206180965724}

### Get diagnostics on convergence / function behaviour for free

- could also specify logging and look at this live in longer problems

In [12]:
from estimagic import criterion_plot, params_plot

In [13]:
criterion_plot(result)

In [14]:
params_plot(result)